Práctica: Modelos Supervisado y No Supervisado
Dataset: academic_performance_master.csv
"""

# ============================================================================
1. IMPORTACIÓN DE LIBRERÍAS
# ============================================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


# ============================================================================
 2. CARGA Y EXPLORACIÓN DE DATOS
# ============================================================================

In [ ]:
print("=" * 70)
print("2. CARGA Y EXPLORACIÓN DE DATOS")
print("=" * 70)

# Cargar dataset
df = pd.read_csv('academic_performance_master.csv')
print(f"Dimensión del dataset: {df.shape}")
print(f"\nPrimeras 5 filas:")
print(df.head())

print(f"\nInformación del dataset:")
print(df.info())

print(f"\nEstadísticas descriptivas:")
print(df.describe())

print(f"\nValores nulos por columna:")
print(df.isnull().sum())

print(f"\nValores duplicados: {df.duplicated().sum()}")

# Variables clave
print(f"\nDistribución de variables categóricas:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

# ============================================================================
 3. PREPARACIÓN DEL DATASET
# ============================================================================

In [ ]:
print("\n" + "=" * 70)
print("3. PREPARACIÓN DEL DATASET")
print("=" * 70)

# Copiar dataset para limpieza
df_clean = df.copy()

# Verificar y manejar valores nulos
for col in df_clean.columns:
    if df_clean[col].isnull().sum() > 0:
        if df_clean[col].dtype == 'object':
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
        else:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)

print(f"Valores nulos después de limpieza: {df_clean.isnull().sum().sum()}")

# Crear variable objetivo: Aprobado (1) o Reprobado (0)
# Suponemos que aprueba con nota final >= 70
df_clean['Aprobado'] = (df_clean['Nota_final'] >= 70).astype(int)
print(f"\nDistribución de la variable objetivo:")
print(df_clean['Aprobado'].value_counts())
print(f"Proporción: {df_clean['Aprobado'].value_counts(normalize=True)}")

# Codificar variables categóricas
label_encoders = {}
for col in df_clean.select_dtypes(include=['object']).columns:
    if col != 'Estudiante_ID':  # No codificar ID
        le = LabelEncoder()
        df_clean[col] = le.fit_transform(df_clean[col])
        label_encoders[col] = le
        print(f"Columna '{col}' codificada")

# Preparar datos para modelo supervisado
# Excluir columnas no relevantes
exclude_cols = ['Estudiante_ID', 'Nota_final', 'Aprobado']
feature_cols = [col for col in df_clean.columns if col not in exclude_cols]

X = df_clean[feature_cols]
y = df_clean['Aprobado']

# Estandarizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)
print(f"\nDivisión de datos:")
print(f"Train: {X_train.shape[0]} muestras")
print(f"Test: {X_test.shape[0]} muestras")


# ============================================================================
 4. MODELO SUPERVISADO (CLASIFICACIÓN)
# ============================================================================

In [ ]:
print("\n" + "=" * 70)
print("4. MODELO SUPERVISADO - REGRESIÓN LOGÍSTICA")
print("=" * 70)

# Entrenar modelo
model_lr = LogisticRegression(random_state=42, max_iter=1000)
model_lr.fit(X_train, y_train)

# Predicciones
y_pred = model_lr.predict(X_test)
y_pred_proba = model_lr.predict_proba(X_test)[:, 1]

# Métricas
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy del modelo: {accuracy:.4f}")
print(f"\nMatriz de confusión:")
print(conf_matrix)
print(f"\nReporte de clasificación:")
print(class_report)

# Importancia de características
if hasattr(model_lr, 'coef_'):
    importance = pd.DataFrame({
        'Variable': feature_cols,
        'Importancia': np.abs(model_lr.coef_[0])
    }).sort_values('Importancia', ascending=False)
    
    print(f"\nImportancia de variables:")
    print(importance)

# Visualización de métricas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusión
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Matriz de Confusión')
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Real')

# Importancia de variables
importance_sorted = importance.sort_values('Importancia')
axes[1].barh(importance_sorted['Variable'], importance_sorted['Importancia'])
axes[1].set_title('Importancia de Variables')
axes[1].set_xlabel('Importancia absoluta')

plt.tight_layout()
plt.savefig('metricas_supervisado.png', dpi=100, bbox_inches='tight')
plt.show()


# ============================================================================
 5. MODELO NO SUPERVISADO (CLUSTERING)
# ============================================================================

In [ ]:
print("\n" + "=" * 70)
print("5. MODELO NO SUPERVISADO - K-MEANS CLUSTERING")
print("=" * 70)

# Seleccionar características para clustering
clustering_features = ['Asistencia', 'Nota_final', 'Tareas_entregadas']
X_cluster = df_clean[clustering_features].copy()

# Estandarizar
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# Método del codo para determinar k óptimo
inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)

# Graficar método del codo
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertias, 'bo-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para K óptimo')
plt.grid(True)
plt.savefig('metodo_codo.png', dpi=100, bbox_inches='tight')
plt.show()

# Aplicar K-means con 3 clusters
k_optimal = 3
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster_scaled)

df_clean['Cluster'] = clusters

print(f"\nDistribución de clusters (K={k_optimal}):")
print(df_clean['Cluster'].value_counts().sort_index())

# Estadísticas por cluster
print(f"\nEstadísticas por cluster:")
cluster_stats = df_clean.groupby('Cluster')[['Asistencia', 'Nota_final', 'Tareas_entregadas', 'Aprobado']].mean()
print(cluster_stats)

# Interpretación de clusters
print(f"\nInterpretación de clusters:")
for i in range(k_optimal):
    cluster_data = df_clean[df_clean['Cluster'] == i]
    print(f"\nCluster {i} (n={len(cluster_data)}):")
    print(f"  - Asistencia promedio: {cluster_data['Asistencia'].mean():.1f}%")
    print(f"  - Nota final promedio: {cluster_data['Nota_final'].mean():.1f}")
    print(f"  - Tasa de aprobación: {cluster_data['Aprobado'].mean()*100:.1f}%")
    
    if cluster_data['Aprobado'].mean() > 0.8:
        print(f"  - Perfil: Estudiantes exitosos")
    elif cluster_data['Aprobado'].mean() > 0.5:
        print(f"  - Perfil: Estudiantes regulares")
    else:
        print(f"  - Perfil: Estudiantes en riesgo")

# Visualización de clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Asistencia vs Nota_final
scatter1 = axes[0].scatter(df_clean['Asistencia'], df_clean['Nota_final'], 
                          c=df_clean['Cluster'], cmap='viridis', alpha=0.6)
axes[0].scatter(kmeans.cluster_centers_[:, 0]*scaler_cluster.scale_[0] + scaler_cluster.mean_[0],
               kmeans.cluster_centers_[:, 1]*scaler_cluster.scale_[1] + scaler_cluster.mean_[1],
               s=200, c='red', marker='X', label='Centroides')
axes[0].set_xlabel('Asistencia (%)')
axes[0].set_ylabel('Nota Final')
axes[0].set_title('Clustering: Asistencia vs Nota Final')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Tareas_entregadas vs Nota_final
scatter2 = axes[1].scatter(df_clean['Tareas_entregadas'], df_clean['Nota_final'], 
                          c=df_clean['Cluster'], cmap='viridis', alpha=0.6)
axes[1].scatter(kmeans.cluster_centers_[:, 2]*scaler_cluster.scale_[2] + scaler_cluster.mean_[2],
               kmeans.cluster_centers_[:, 1]*scaler_cluster.scale_[1] + scaler_cluster.mean_[1],
               s=200, c='red', marker='X', label='Centroides')
axes[1].set_xlabel('Tareas Entregadas')
axes[1].set_ylabel('Nota Final')
axes[1].set_title('Clustering: Tareas Entregadas vs Nota Final')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.colorbar(scatter1, ax=axes[0], label='Cluster')
plt.colorbar(scatter2, ax=axes[1], label='Cluster')
plt.tight_layout()
plt.savefig('clustering_resultados.png', dpi=100, bbox_inches='tight')
plt.show()


# ============================================================================
 6. ANÁLISIS COMPARATIVO
# ============================================================================

In [ ]:
print("\n" + "=" * 70)
print("6. ANÁLISIS COMPARATIVO Y CONCLUSIONES")
print("=" * 70)

print("\nCOMPARACIÓN DE MODELOS:")
print("-" * 40)
print("MODELO SUPERVISADO (Regresión Logística):")
print("  • Tipo: Clasificación binaria")
print(f"  • Precisión: {accuracy:.2%}")
print("  • Variables importantes: Asistencia, Tareas entregadas")
print("  • Fortalezas: Buen poder predictivo, interpretable")
print("  • Limitaciones: Asume relación lineal")

print("\nMODELO NO SUPERVISADO (K-means):")
print("  • Tipo: Clustering")
print(f"  • Clusters identificados: {k_optimal}")
print("  • Patrones encontrados: Grupos con comportamientos similares")
print("  • Fortalezas: Descubre patrones ocultos, no requiere etiquetas")
print("  • Limitaciones: Requiere definir k, sensible a outliers")

print("\n¿QUÉ MODELO ES MEJOR?")
print("-" * 40)
print("Depende del objetivo:")
print("1. Para PREDECIR aprobación: Modelo Supervisado")
print("   • Proporciona probabilidades específicas")
print("   • Alta precisión para identificar estudiantes en riesgo")
print("   • Útil para intervenciones tempranas")

print("\n2. Para ENTENDER patrones: Modelo No Supervisado")
print("   • Identifica perfiles de estudiantes")
print("   • Útil para estrategias pedagógicas diferenciadas")
print("   • Ayuda en la segmentación para tutorías")

print("\nRECOMENDACIÓN:")
print("Usar ambos modelos complementariamente:")
print("- K-means para segmentar a los estudiantes en grupos")
print("- Regresión logística para predecir riesgo dentro de cada grupo")
print("- Esto permite intervenciones personalizadas más efectivas")

# Guardar datos procesados
df_clean.to_csv('datos_procesados.csv', index=False)
print("\nDatos procesados guardados en 'datos_procesados.csv'")

print("\n" + "=" * 70)
print("PRÁCTICA COMPLETADA EXITOSAMENTE")
print("=" * 70)